In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 970M (CNMeM is disabled, cuDNN not available)


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

In [4]:
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,1,28,28)
test = test.values.reshape(-1,1,28,28)

In [5]:
Y_train = to_categorical(Y_train, num_classes = 10)

# set random seed for network, maybe not useful
from numpy.random import seed 
seed(42)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print(X_train.shape, Y_train.shape)

(37800, 1, 28, 28) (37800, 10)


In [6]:
model = Sequential()

# conv1
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# conv2
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# conv3
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


# flatten
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(10, activation = "softmax"))
# Compile the model

opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
print("compile done")

compile done


In [7]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

/home/jac/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3 or 4 channels on axis 1. However, it was passed an array with shape (37800, 1, 28, 28) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [8]:
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.6, 
                                            min_lr=0.00001)
    
best_model_path = 'deep_best_m_v2.h5'
best_model = ModelCheckpoint(filepath=best_model_path, monitor='val_loss', save_best_only=True, verbose=1)
epochs = 80
steps_per_epoch = 100
batch_size = 64
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=steps_per_epoch,
                              callbacks=[best_model,learning_rate_reduction])

Epoch 1/80
Epoch 00000: val_loss improved from inf to 0.53503, saving model to best_m_v2.h5
10s - loss: 1.7223 - acc: 0.3739 - val_loss: 0.5350 - val_acc: 0.8181
Epoch 2/80
Epoch 00001: val_loss improved from 0.53503 to 0.29625, saving model to best_m_v2.h5
10s - loss: 0.6501 - acc: 0.7853 - val_loss: 0.2962 - val_acc: 0.9086
Epoch 3/80
Epoch 00002: val_loss improved from 0.29625 to 0.19683, saving model to best_m_v2.h5
9s - loss: 0.4460 - acc: 0.8600 - val_loss: 0.1968 - val_acc: 0.9343
Epoch 4/80
Epoch 00003: val_loss improved from 0.19683 to 0.17636, saving model to best_m_v2.h5
10s - loss: 0.3525 - acc: 0.8852 - val_loss: 0.1764 - val_acc: 0.9433
Epoch 5/80
Epoch 00004: val_loss did not improve
9s - loss: 0.3209 - acc: 0.9009 - val_loss: 0.1887 - val_acc: 0.9367
Epoch 6/80
Epoch 00005: val_loss improved from 0.17636 to 0.13464, saving model to best_m_v2.h5
10s - loss: 0.3115 - acc: 0.9000 - val_loss: 0.1346 - val_acc: 0.9564
Epoch 7/80
Epoch 00006: val_loss did not improve
9s - los

Epoch 00058: val_loss did not improve
10s - loss: 0.0602 - acc: 0.9800 - val_loss: 0.0375 - val_acc: 0.9890
Epoch 60/80
Epoch 00059: val_loss did not improve
9s - loss: 0.0704 - acc: 0.9795 - val_loss: 0.0388 - val_acc: 0.9879
Epoch 61/80
Epoch 00060: val_loss did not improve
9s - loss: 0.0800 - acc: 0.9772 - val_loss: 0.0421 - val_acc: 0.9864
Epoch 62/80
Epoch 00061: val_loss did not improve
8s - loss: 0.0679 - acc: 0.9803 - val_loss: 0.0404 - val_acc: 0.9867
Epoch 63/80
Epoch 00062: val_loss did not improve
8s - loss: 0.0660 - acc: 0.9791 - val_loss: 0.0423 - val_acc: 0.9869
Epoch 64/80
Epoch 00063: val_loss did not improve
9s - loss: 0.0662 - acc: 0.9803 - val_loss: 0.0401 - val_acc: 0.9881
Epoch 65/80
Epoch 00064: val_loss did not improve

Epoch 00064: reducing learning rate to 0.00021600000327453016.
9s - loss: 0.0813 - acc: 0.9753 - val_loss: 0.0365 - val_acc: 0.9890
Epoch 66/80
Epoch 00065: val_loss did not improve
9s - loss: 0.0613 - acc: 0.9816 - val_loss: 0.0357 - val_acc: 0.

In [9]:
model = load_model(best_model_path)  # try callback model
# predict results
results = model.predict(test)
# select the indix with the maximum probability
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
import arrow
ts = str(arrow.now())[:-10].replace(':','_').replace('.',"_")
res_file = "deep_play_cnn_mnist_datagen_{}.csv".format(ts)
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv(res_file,index=False)
print('save done',res_file)

save done deep_play_cnn_mnist_datagen_2017-09-07T10_30_43_60.csv
